### Step 1: Setting Up the Python Application

Import all the dependencies

In [27]:
# Imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_chroma import Chroma
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

import pandas as pd
import json

Define some constants for the project

In [ ]:
# Model name
MODEL_NAME = "gpt-3.5-turbo"

# Define the API key for OpenAI
OPENAI_API_KEY = "TO_BE_FILLED"

### Step 2: Generating Real Estate Listings

In [ ]:
def generate_real_state_listings(count=10):
    llm = ChatOpenAI(
        model_name=MODEL_NAME, 
        temperature=0.0, 
        openai_api_key=OPENAI_API_KEY
    )

    # Define the prompt template for generating real estate listings
    prompt_template = PromptTemplate(
        input_variables=["listing_count"],
        template="""
        Generate {listing_count} real estate listings in JSON format. Each listing should have the following fields:
            - Neighborhood
            - Price
            - Bedrooms
            - Bathrooms
            - House Size
            - Description
            - Neighborhood Description
    
        Here is an example of the JSON format:
        [
            {{
                "Neighborhood": "Green Oaks",
                "Price": "$800,000",
                "Bedrooms": 3,
                "Bathrooms": 2,
                "House Size": "2,000 sqft",
                "Description": "Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.",
                "Neighborhood Description": "Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze."
            }},
            ...
        ]
        
        Please provide the listings in this exact JSON format.
        """
    )

    # Generate the listings by calling the model
    prompt = prompt_template.format(listing_count=count)
    response = llm.invoke(prompt)

    # Parse the JSON response
    listings = json.loads(response.content.strip())

    # Create a DataFrame and save it to a CSV file
    df = pd.DataFrame(listings)
    df.to_csv('real_estate_listings.csv', index=False)

    print("CSV file 'real_estate_listings.csv' generated successfully.")

In [ ]:
generate_real_state_listings(10)

### Step 3: Storing Listings in a Vector Database

In [ ]:
def load():
    # load the documents
    loader = CSVLoader(file_path="./real_estate_listings.csv")
    documents = loader.load()
    
    # split it into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    split_docs = text_splitter.split_documents(documents)
    
    # extract metadata from the CSV and update documents
    for doc in split_docs:
        row_data = doc.page_content.split('\n')

        metadata_dict = {}
        for line in row_data:
            key, value = line.split(': ', 1)
            metadata_dict[key.strip()] = value.strip()
        doc.metadata.update(metadata_dict)
        
    return split_docs

In [ ]:
# load the documents and add metadata
documents = load()

# create the open-source embedding function
embedding_function = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# create simple ids
ids = [str(i) for i in range(1, len(documents) + 1)]

# load it into Chroma
db = Chroma.from_documents(documents, embedding_function, ids=ids)

### Step 4: Building the User Preference Interface

In [ ]:
questions = [   
    "How big do you want your house to be?",
    "What are 3 most important things for you in choosing this property?",
    "Which amenities would you like?",
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?"
]

answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

In [ ]:
def search(query, top_k=3):
    return db.similarity_search(query, k=top_k)

In [31]:
# combine user preferences into a single query string
query = "\n".join(answers)

# perform the search
search_results = search(query)

for result in search_results:
    print(result.metadata)

{'Bathrooms': '2', 'Bedrooms': '3', 'Description': 'Welcome to this charming 3-bedroom, 2-bathroom home in the desirable Maple Grove neighborhood. The open floor plan features a bright and airy living room with hardwood floors and a cozy fireplace. The updated kitchen boasts granite countertops and stainless steel appliances. Retreat to the master suite with a walk-in closet and en-suite bathroom. Enjoy outdoor living in the private backyard with a patio and garden. Experience comfort and convenience in Maple Grove.', 'House Size': '2,000 sqft', 'Neighborhood': 'Maple Grove', 'Neighborhood Description': 'Maple Grove is a quiet and friendly neighborhood with tree-lined streets and parks. Residents enjoy easy access to shopping centers, restaurants, and schools. Explore the nearby Maple Grove Park for outdoor recreation or take a short drive to the Maple Grove Shopping Center.', 'Price': '$550,000', 'row': 4, 'source': './real_estate_listings.csv'}
{'Bathrooms': '3', 'Bedrooms': '4', 'De

### Step 6: Personalizing Listing Descriptions

In [44]:
# generate listing description
def generate_listing_description(listing):
    return f"""
        Neighborhood: {listing["Neighborhood"]}
        Price: {listing["Price"]}
        Bedrooms: {listing["Bedrooms"]}
        Bathrooms: {listing["Bathrooms"]}
        House Size: {listing["House Size"]}
        Description: {listing["Description"]}
        Neighborhood Description: {listing["Neighborhood Description"]}
        """

In [33]:
# generate user preferences according to the questions and answers
def generate_user_preferences():
    questions_and_answers = [f"{question}\n {answer}\n" for question, answer in zip(questions, answers)]
    return "\n".join(questions_and_answers)

In [46]:
def generate_tailored_result(preferences, listing):
    listing_description = generate_listing_description(listing)
    
    prompt = prompt_template.format(
        listing_description=listing_description,
        preferences=preferences
    )
    
    print(prompt)
    
    llm = ChatOpenAI(
        model_name=MODEL_NAME, 
        temperature=0.0, 
        openai_api_key=OPENAI_API_KEY
    )
    
    response = llm.invoke(prompt)
    tailored_description = response.content.strip()
    
    print(f"Tailored Description:\n{tailored_description}\n")

In [35]:
# generate user preferences according to the questions and answers
def generate_user_preferences():
    questions_and_answers = [f"{question}\n {answer}\n" for question, answer in zip(questions, answers)]
    return "\n".join(questions_and_answers)

In [48]:
# combine questions and answers into a single string
preferences = generate_user_preferences()

# define the prompt template for generating tailored descriptions
prompt_template = PromptTemplate(
    input_variables=["listing", "preferences"],
    template="""
    Based on the following user preferences: 
    {preferences}
    
    Generate a tailored description for the buyer. This involves subtly emphasizing aspects of the property that align with what the buyer is looking for. Ensure that the augmentation process enhances the appeal of the listing without altering factual information.

    Listing:
    {listing_description}

    Tailored Description:
    """
)
    
# Generate tailored descriptions for search results
for result in search_results:
    generate_tailored_result(preferences, result.metadata)


    Based on the following user preferences: 
    How big do you want your house to be?
 A comfortable three-bedroom house with a spacious kitchen and a cozy living room.

What are 3 most important things for you in choosing this property?
 A quiet neighborhood, good local schools, and convenient shopping options.

Which amenities would you like?
 A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.

Which transportation options are important to you?
 Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.

How urban do you want your neighborhood to be?
 A balance between suburban tranquility and access to urban amenities like restaurants and theaters.

    
    Generate a tailored description for the buyer. This involves subtly emphasizing aspects of the property that align with what the buyer is looking for. Ensure that the augmentation process enhances the appeal of the listing without altering factual informa